In [1]:
import numpy as np
import pandas as pd
import os
from numpy.random import Generator as gen
from numpy.random import PCG64 as pcg
import matplotlib.pyplot as plt
from  datetime import datetime

np.set_printoptions(suppress=True, linewidth=100, precision=2)

DIR_RESOURCES = 'data_analyst\Resources\Data'
dir_sources = os.path.join(os.getcwd(),DIR_RESOURCES)

In [2]:
def clean_up_filename(name):
    chars_to_replace = '-* '
    trans_table = str.maketrans(chars_to_replace,'_' * len(chars_to_replace))
    return os.path.splitext(name.strip().translate(trans_table))[0]
# use it only for debugging and setting up project    
# some files might require manual manipulation
[print(f'{clean_up_filename(f)} = np.genfromtxt(os.path.join(dir_sources,\'{f}\'))') for f in os.listdir(dir_sources) if 'loan' in f]

loan_data_dictionary = np.genfromtxt(os.path.join(dir_sources,'loan-data-dictionary.xlsx'))
loan_data = np.genfromtxt(os.path.join(dir_sources,'loan-data.csv'))
loan_EUR_USD = np.genfromtxt(os.path.join(dir_sources,'loan_EUR-USD.csv'))


[None, None, None]

In [3]:
class DataCleanerColumn:
    def __init__(self, name, data , index, dtype):
        self.index = index
        self.name = name
        self.data = data
        self.dtype = dtype

    def unique(self,return_counts=False):
        return np.unique(self.data,return_counts=return_counts)
        
    def __str__(self):
        return (f'{col.dtype} with name {col.name} and index {col.index}')

In [4]:
class DataCleaner:
    def __iter__(self):
        return self
    def __next__(self) -> DataCleanerColumn:
        while self._iter_index_type < len(self.supported_dtypes_list):
            #print(self._iter_index_type,self._iter_index_column)
            dtype = self.supported_dtypes_list[self._iter_index_type]
            shp = self._data[dtype][2].shape
            dim = self._data[dtype][2].ndim
            has_any = (shp[0] > 0)
            is_one_dimension_and_zero = (self._iter_index_column == 0)
            is_2d_and_index_within_bounds = ((dim > 1) and (self._iter_index_column < shp[1]))
            if has_any and (is_one_dimension_and_zero or is_2d_and_index_within_bounds):
                col = self[dtype,self._iter_index_column]
                self._iter_index_column += 1
                return col
            #print(self._iter_index_type)
            self._iter_index_column = 0
            self._iter_index_type += 1

        self._iter_index_column = 0
        self._iter_index_type = 0
        raise StopIteration

    def __show_basic_info(self):
        # method acts weird, I am not aware of the shunanigans here
        self._iter_index_type = 0
        self._iter_index_column = 0
        for col in self:
            print(col)
                
    def __init__(self,file_path,delimiter,generate=False,restore=False,usecols=None):
        self.supported_dtypes_list = [
            float,
            str,
            np.datetime64,
            int
        ]
        
        self._iter_index_type = 0
        self._iter_index_column = 0
        
        self._data = {}
        for dtype in self.supported_dtypes_list:
            self._data[dtype] = [
                                    np.array([],dtype=int),
                                    np.array([],dtype=str),
                                    np.array([],dtype=dtype)
                                ]
            
        self.file_path = file_path
        self.file_delimiter = delimiter
        
        if (generate and restore):
            raise Exception("Both kwargs 'generate' and 'restore' can't be assigned to True")
        if generate:
            self.gen_data(usecols)
        if restore:
            self.restore(file_path)

    def restore(self,name):
        data = np.load(name)
        for dtype in self.supported_dtypes_list:
            base_name = str(dtype).strip().replace(' ','').replace('\'','')
            self._data[dtype][0] = data[base_name + '_column_original_index']
            self._data[dtype][1] = data[base_name + '_header_name']
            self._data[dtype][2] = data[base_name + '_data']
            
    def export(self,name):
        dict_arrays = {}
        for dtype in self.supported_dtypes_list:
            base_name = str(dtype).strip().replace(' ','').replace('\'','')
            dict_arrays[base_name + '_column_original_index'] = self._data[dtype][0]
            dict_arrays[base_name + '_header_name'] = self._data[dtype][1]
            dict_arrays[base_name + '_data'] = self._data[dtype][2]
        print(dict_arrays.keys())
        np.savez(name,**dict_arrays)

    def rename_column(self, dtype, index, new_name):
        self._data[dtype][1][index] = new_name
        
    def delete_column(self, dtype, index):
        self._data[dtype][0] = np.delete(self._data[dtype][0],index,axis=None)
        self._data[dtype][1] = np.delete(self._data[dtype][1],index,axis=None)
        self._data[dtype][2] = np.delete(self._data[dtype][2],index,axis=1)

    def add_column(self, dtype, nparray, ori_index, name):
        self._data[dtype][0] = np.append(self._data[dtype][0], ori_index)
        self._data[dtype][1] = np.append(self._data[dtype][1], name)
        try:
            self._data[dtype][2] = np.column_stack([self._data[dtype][2],nparray]).astype(dtype=dtype)
        except Exception as e:
            if self._data[dtype][2].size == 0:
                self._data[dtype][2] = nparray.copy().astype(dtype=dtype)
            else:
                raise e
                
    def move_column(self, from_dtype, from_index, to_dtype):
        ori_index = self._data[from_dtype][0][from_index]
        from_name = self._data[from_dtype][1][from_index]
        from_data = self._data[from_dtype][2][:,from_index]

        self.add_column(to_dtype, from_data, ori_index, from_name)
        self.delete_column(from_dtype,from_index)

    def try_datetime(self, nparray, datetime_format):
        # datetime lib handles more format options like:
        # '%b-%y' (Mmm-YY : May-15)
        unique_dates = np.unique(nparray)
        array_dates = [np.datetime64('NaT') 
                       if date_str == '' else np.datetime64(datetime.strptime(date_str,datetime_format)) 
                       for date_str in unique_dates]
        cpy = nparray.copy()
        for i in range(unique_dates.shape[0]):
            cpy[cpy == unique_dates[i]] = array_dates[i]
        # now is safe to overwrite
        nparray[:] = cpy[:]
        return cpy

    def get_type_data(self, dtype):
        return self._data[dtype][2]
        
    def __getitem__(self, key) -> DataCleanerColumn:
        dtype,index = key
        name = self._data[dtype][1][index]
        try:
            data = self._data[dtype][2][:,index]
        except IndexError as e:
            data = self._data[dtype][2][:]
        return DataCleanerColumn(name, data, index, dtype)
    
    def gen_data(self,usecols=None):
        data_nan = np.genfromtxt(self.file_path,
                                 delimiter=self.file_delimiter,
                                 skip_header=True,
                                 usecols=usecols)
        # first pass
        # open only numeric data
        # calculate max val as placeholder for NaN
        # identify non-numeric columns (where calculated mean is NaN)
        temporary_max_val_plus_one = np.nanmax(data_nan) + 1
        temp_mean = np.atleast_1d(np.nanmean(data_nan,axis=0))
        
        self.temp_stats = np.array([np.atleast_1d(np.nanmin(data_nan,axis = 0)),
                                  temp_mean,
                                  np.atleast_1d(np.nanmax(data_nan, axis = 0))])

        self._data[str][0] = np.atleast_1d(np.argwhere(np.isnan(temp_mean)).squeeze())
        self._data[float][0] = np.atleast_1d(np.argwhere(~np.isnan(temp_mean)).squeeze())

        npa_usecols = np.atleast_1d(usecols)
        if usecols is None:
            npa_usecols = np.atleast_1d(np.arange(0, data_nan.shape[0]))
            
        # second pass
        # STR
        if self._data[str][0].shape[0] > 0:
            # get source index of columns if usecols is passed
            self._data[str][0] = npa_usecols[self._data[str][0]]
            self._data[str][2] = np.genfromtxt(self.file_path,
                                          delimiter=self.file_delimiter,
                                          dtype=str,
                                          usecols=self._data[str][0],
                                          skip_header=True)

        # FLOAT
        if self._data[float][0].shape[0] > 0:
            # get source index of columns if usecols is passed
            self._data[float][0] = npa_usecols[self._data[float][0]]
            self._data[float][2] = np.genfromtxt(self.file_path,
                                          dtype='float',
                                          delimiter=self.file_delimiter,
                                          usecols=self._data[float][0],
                                          skip_header=True,
                                          filling_values=temporary_max_val_plus_one)

        # this works because the above data contains all rows minus header
        skip_footer_count = max(self._data[str][2].shape[0], self._data[float][2].shape[0])
        headers_all = np.genfromtxt(self.file_path,
                                    delimiter=self.file_delimiter,
                                    dtype=str,
                                    skip_footer=skip_footer_count)
        
        self._data[str][1] = headers_all[self._data[str][0]]
        self._data[float][1]= headers_all[self._data[float][0]]

    def __str__(self):
        output = ''
        for dtype in self.supported_dtypes_list:
            output += f'\nSUMMARY FOR {str(dtype)}'
            output += f'\nColumn original index:\n{self._data[dtype][0]}'
            output += f'\nColumn name:\n{self._data[dtype][1]}'
            output += f'\nData:\n{self._data[dtype][2]}\n'
        return output

In [5]:
# Sources
#df_xl = pd.read_excel(os.path.join(dir_sources,'loan-data-dictionary.xlsx'))
loan_data_cleaner = DataCleaner(os.path.join(dir_sources, 'loan-data.csv') ,delimiter=';',generate=True)
#loan_curr_cleaner = DataCleaner(os.path.join(dir_sources,'loan_EUR-USD.csv') ,delimiter=',',generate=True)

C:\Users\user\AppData\Local\Temp\ipykernel_8236\1876882789.py:142: RuntimeWarning: Mean of empty slice
  temp_mean = np.atleast_1d(np.nanmean(data_nan,axis=0))
C:\Users\user\AppData\Local\Temp\ipykernel_8236\1876882789.py:144: RuntimeWarning: All-NaN slice encountered
  self.temp_stats = np.array([np.atleast_1d(np.nanmin(data_nan,axis = 0)),
C:\Users\user\AppData\Local\Temp\ipykernel_8236\1876882789.py:146: RuntimeWarning: All-NaN slice encountered
  np.atleast_1d(np.nanmax(data_nan, axis = 0))])


In [6]:
# checkpoint
#loan_data_cleaner.export('loan_data_imported')
#loan_data_cleaner.restore('loan_data_imported.npz')

In [7]:
print(loan_data_cleaner)
#loan_data_cleaner.export('loan_data_imported')
#loan_data_cleaner = DataCleaner('loan_data_imported.npz' ,delimiter=';',restore=True)


SUMMARY FOR <class 'float'>
Column original index:
[ 0  2  4  6  7 13]
Column name:
['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']
Data:
[[48010226.      35000.      35000.         13.33     1184.86     9452.96]
 [57693261.      30000.      30000.   68616520.        938.57     4679.7 ]
 [59432726.      15000.      15000.   68616520.        494.86     1969.83]
 ...
 [50415990.      10000.      10000.   68616520.   68616520.       2185.64]
 [46154151.   68616520.      10000.         16.55      354.3      3199.4 ]
 [66055249.      10000.      10000.   68616520.        309.97      301.9 ]]

SUMMARY FOR <class 'str'>
Column original index:
[ 1  3  5  8  9 10 11 12]
Column name:
['issue_d' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']
Data:
[['May-15' 'Current' ' 36 months' ... 'Verified'
  'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226' 'CA']
 ['' 'Current' ' 36 months' ... 'Source Verified'
  'https://ww

In [8]:
# issue_d
loan_data_cleaner.try_datetime(loan_data_cleaner[str,0].data,'%b-%y')
loan_data_cleaner.move_column(str, 0, np.datetime64)
loan_data_cleaner[np.datetime64,0].unique()

array(['2015-01-01T00:00:00.000000', '2015-02-01T00:00:00.000000', '2015-03-01T00:00:00.000000',
       '2015-04-01T00:00:00.000000', '2015-05-01T00:00:00.000000', '2015-06-01T00:00:00.000000',
       '2015-07-01T00:00:00.000000', '2015-08-01T00:00:00.000000', '2015-09-01T00:00:00.000000',
       '2015-10-01T00:00:00.000000', '2015-11-01T00:00:00.000000', '2015-12-01T00:00:00.000000',
                              'NaT'], dtype='datetime64[us]')

In [9]:
# loan_status column - translate to 0 good status, 1 bad status
col = loan_data_cleaner[str,0]
col.unique()
# manual list of what we decide is bad status
list_bad_status = ['', 'Charged Off', 'Default', 'Late (31-120 days)']
col.data[:] = np.where(np.isin(col.data, list_bad_status), 1,0)
col.unique()

array(['0', '1'], dtype='<U69')

In [10]:
# term column
col = loan_data_cleaner[str,1].data
np.unique(col)
col[:] = np.chararray.replace(col,'months','')
col[:]  = np.chararray.strip(col)
default_term = 60
col[col == ''] = default_term
loan_data_cleaner.rename_column(str, 1,'term_months')
np.unique(col)

array(['36', '60'], dtype='<U69')

In [11]:
# grade column
# we have sub-grade column with more details, delete this column
# before, make sure any row with sub-grade empty, is filled with grade value
col_g = loan_data_cleaner[str,2].data
np.unique(col_g)
col_sg = loan_data_cleaner[str,3].data
print(f'Count unique values grade col {np.unique(col_g,return_counts=True)[1]}')
print(f'Count unique values sub_grade col {np.unique(col_sg,return_counts=True)[1]}')
# if sub_grade is empty, we will fill with grade + 5 (A5,B5,...)
default_undefined_grade_index = np.array('5')
col_sg[:] = np.where( (col_sg == '') & (col_g != ''), np.char.add(col_g, default_undefined_grade_index), col_sg)
# delete grade column
loan_data_cleaner.delete_column(str, 2)
col_sg = loan_data_cleaner[str,2].data
print(np.unique(col_sg,return_counts=True)[0])
print(np.unique(col_sg,return_counts=True)[1])
# any empty value in sub grades, we will fill it with arbitrary value
filling_subgrade = 'Z1'
col_sg[:] = np.where(col_sg == '', filling_subgrade, col_sg)

Count unique values grade col [ 515 1632 2606 2766 1389  816  236   40]
Count unique values sub_grade col [514 285 278 239 323 502 509 517 530 553 494 629 567 586 564 423 391 267 250 255 223 235 162 171
 139 114  94  52  34  43  16  19  10   3   7   2]
['' 'A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4'
 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']
[  9 285 278 239 323 592 509 517 530 553 633 629 567 586 564 577 391 267 250 255 288 235 162 171
 139 160  94  52  34  43  24  19  10   3   7   5]


In [12]:
# sub-grade column
col = loan_data_cleaner[str,2].data
np.unique(col)
if_value = ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5']
if_value.append(filling_subgrade)
if_value.sort()
set_value = [(i + 1) for i in range(len(if_value))]
for i in range(len(if_value)):
    col[col == if_value[i]] = set_value[i]
col
np.unique(col)

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

In [13]:
# verification_status column
col = loan_data_cleaner[str,3].data
np.unique(col)
unverified_values = ['', 'Not Verified']
col[:] = np.where(np.isin(col,unverified_values),0,1)
np.unique(col)

array(['0', '1'], dtype='<U69')

In [14]:
# url column - useless, loan_id has same values and the url is unique
col = loan_data_cleaner[str,4].data
np.unique(col)
base_url = 'https://www.lendingclub.com/browse/loanDetail.action?loan_id='
loan_data_cleaner.delete_column(str, 4)

In [15]:
# addr_state column pre-analysis
# looking at the amount of missing values ('')
# it's above 45 states, so any coefficient for small states will be altered by outliers
# Instead we will group states by region (S,W,N,E)
col = loan_data_cleaner[str,4].data
idx = np.argwhere(col == '')
s_name, s_count = np.unique(col,return_counts=True)
s_count_sorted = np.argsort(-s_count)
s_name[s_count_sorted],s_count[s_count_sorted]
total_missing_info = s_count[np.argwhere(s_name == '')][0][0]
print(f'Perc of records without data {total_missing_info / s_count.sum()}')
print(f'Records with \'\' value is the top {round(np.percentile(s_count,total_missing_info / s_count.sum() * 100),2)}%')
s_count[s_count_sorted],s_count[s_count_sorted] / s_count.sum()

Perc of records without data 0.05
Records with '' value is the top 20.15%


(array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10], dtype=int64),
 array([0.13, 0.08, 0.08, 0.07, 0.05, 0.04, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.02, 0.02, 0.02,
        0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
        0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  ]))

In [16]:
# add region column
col = loan_data_cleaner[str,4].data
np.unique(col)
west = ['AK', 'CA', 'HI', 'NV', 'OR', 'WA']
east = ['CT', 'DE', 'DC', 'FL', 'GA', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'NC', 'PA', 'RI', 'SC',
        'VT', 'VA', 'WV']
north = ['ID', 'IL', 'IN', 'IA', 'KS', 'MI', 'MN', 'MO', 'MT', 'NE', 'ND', 'OH', 'SD', 'WI', 'WY']
south = ['AL', 'AR', 'AZ', 'CO', 'KY', 'LA', 'MS', 'NM', 'OK', 'TN', 'TX', 'UT']
all_states = [west, east, north, south]
# region 0, means undefined (NULL)
# region [1-4], W,E,N,S
for idx, state in enumerate(all_states):
    col[np.isin(col,state)] = (idx + 1)
col[col == ''] = 0
#np.unique(col_cpy)  
loan_data_cleaner.rename_column(str, 4,'Region')

In [17]:
# can all strings now be converted to INT?
loan_data_cleaner._data[str][2].astype(dtype=int)

array([[ 0, 36, 13,  1,  1],
       [ 0, 36,  5,  1,  2],
       [ 0, 36, 10,  1,  2],
       ...,
       [ 0, 36,  5,  1,  1],
       [ 0, 36, 17,  1,  3],
       [ 0, 36,  4,  0,  3]])

In [18]:
# for numeric values we need to decide how to fill the empty cells
# max is not always preferred as it might be bad for the particular column
# for instance, if you have a column with client max loan value and you set it to max
# most likely isn't the best possible value. Instead, min value will be more appropiate for the unknown
loan_data_cleaner.temp_stats.transpose()
print(loan_data_cleaner._data[float][1])
# we set for max for all except funded_amnt
col_fill_method = [np.nanmax] * loan_data_cleaner._data[float][1].size
col_fill_method[2] = np.nanmin
col_fill_method[0] = None
col_fill_method
temp_stats_num = loan_data_cleaner.temp_stats[:,loan_data_cleaner._data[float][0]].transpose()
max_val_plus_one = np.max(loan_data_cleaner.temp_stats[:,loan_data_cleaner._data[float][0]]) + 1
for index, method in enumerate(col_fill_method):
    col = loan_data_cleaner[float, index]
    if method != None:
        val = method(temp_stats_num[index])
        print(f'{col.name} : {val}')
        col.data[col.data == max_val_plus_one] = val
    else:
        print(f'{col.name} - Ignored')

['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']
id - Ignored
loan_amnt : 35000.0
funded_amnt : 1000.0
int_rate : 28.99
installment : 1372.97
total_pymnt : 41913.62


In [19]:
# currency exchnge eur_usd 12 months of data
loan_curr_cleaner = DataCleaner(os.path.join(dir_sources,'loan_EUR-USD.csv') ,delimiter=',',generate=True,usecols=3)

In [20]:
for col in loan_curr_cleaner:
    print(col)

<class 'float'> with name Close and index 0


In [21]:
dates = loan_data_cleaner[np.datetime64,0]
dates_as_int = dates.data.astype('datetime64[M]').astype(int)
temp = np.zeros_like(dates_as_int).astype(dtype='float')
for m in range(12):
    dte = (np.datetime64('2015-01-01','M') + m).astype(int)
    temp[((dates_as_int - dte) == 0)] = loan_curr_cleaner[float,0].data[m]
# use mean ex rate for NaT values    
temp[temp == 0] = np.mean(loan_curr_cleaner[float,0].data)    
np.unique(temp)
loan_data_cleaner.add_column(float,temp,-1,'exchange_rate')

In [22]:
# everything now as numeric data
while(loan_data_cleaner._data[str][2].size > 0):
    loan_data_cleaner.move_column(str,0,float)

In [47]:
# stack header to cleaned data and save output
#loan_data_cleaner.export('loan_data_imported')
#loan_data_cleaner = DataCleaner('loan_data_imported.npz',delimiter=',',restore=True)
data_clean_and_sorted = loan_data_cleaner.get_type_data(float)[np.argsort(loan_data_cleaner[float,0].data)]
data_clean_and_sorted = np.vstack( [loan_data_cleaner._data[float][1], data_clean_and_sorted] )
np.savetxt('loan_data_preprocessed.csv', data_clean_and_sorted, fmt = '%s', delimiter = ',')